In [1]:
from pyspark.sql import SparkSession

# Spark session is auto-created in this image, but confirm config
spark = SparkSession.builder.getOrCreate()

# Sample data: Create a simple DataFrame
data = [("Alice", 34), ("Bob", 45)]
df = spark.createDataFrame(data, ["name", "age"])

# Write to Iceberg table (stored in MinIO's s3://lakehouse/local.my_table/)
df.writeTo("local.my_table").using("iceberg").createOrReplace()

# Verify by reading back
spark.read.table("local.my_table").show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|  Bob| 45|
+-----+---+

